In [1]:
import tkinter as tk
from PIL import Image, ImageTk
from copy import deepcopy
import math
import random
import time

In [2]:
def print_board(board):
    global board_size
    print(' ',*[x for x in range(board_size)])
    for row_index, row_value in enumerate(board):
        print(row_index, *row_value)
        
def init_board():
    global board_size
    global unit_rows
    board = [['-']*board_size for _ in range(board_size)]
    for i in range(board_size):
        for j in range(board_size):
            if (i+j)%2 == 1:
                if i < unit_rows:
                    board[i][j] = 'o'
                elif i >= board_size - unit_rows:
                    board[i][j] = 'x'
                else:
                    board[i][j] = ' '
    return board

In [3]:
# move from a to b
def is_legal_non_jump(board, a, b, turn_player):
    global board_size
    if board[a[0]][a[1]] == 'X' or board[a[0]][a[1]] == 'O':
        if turn_player == 'A':
            step_i = (b[0]-a[0])//abs(b[0]-a[0])
            step_j = (b[1]-a[1])//abs(b[1]-a[1])
            if (0 <= a[0] < board_size and 0 <= a[1] < board_size and 0 <= b[0] < board_size and 0 <= b[1] < board_size
                and board[a[0]][a[1]] == 'X' and board[b[0]][b[1]] == ' ' and abs(b[0]-a[0]) == abs(b[1]-a[1])):
                for i in range(1, abs(b[0]-step_i-a[0])+1):
                    if board[a[0]+i*step_i][a[1]+i*step_j] != ' ':
                        return False
                return True
            else:
                return False
        elif turn_player == 'B':
            step_i = (b[0]-a[0])//abs(b[0]-a[0])
            step_j = (b[1]-a[1])//abs(b[1]-a[1])
            if (0 <= a[0] < board_size and 0 <= a[1] < board_size and 0 <= b[0] < board_size and 0 <= b[1] < board_size
                and board[a[0]][a[1]] == 'O' and board[b[0]][b[1]] == ' ' and abs(b[0]-a[0]) == abs(b[1]-a[1])):
                for i in range(1, abs(b[0]-step_i-a[0])+1):
                    if board[a[0]+i*step_i][a[1]+i*step_j] != ' ':
                        return False
                return True
            else:
                return False
                                        
    if turn_player == 'A':
        if (0 <= a[0] < board_size and 0 <= a[1] < board_size and 0 <= b[0] < board_size and 0 <= b[1] < board_size
            and board[a[0]][a[1]] == 'x' and board[b[0]][b[1]] == ' ' and b[0]-a[0] == -1 and abs(b[1]-a[1]) == 1):
            return True
        else:
            return False
    elif turn_player == 'B':
        if (0 <= a[0] < board_size and 0 <= a[1] < board_size and 0 <= b[0] < board_size and 0 <= b[1] < board_size
            and board[a[0]][a[1]] == 'o' and board[b[0]][b[1]] == ' ' and b[0]-a[0] == 1 and abs(b[1]-a[1]) == 1):
            return True
        else:
            return False

def is_legal_jump(board, a, b, turn_player, is_now_multi_jump=False):
    global board_size
    if board[a[0]][a[1]] == 'X' or board[a[0]][a[1]] == 'O':
        if is_now_multi_jump:
            if turn_player == 'A':
                if (0 <= a[0] < board_size and 0 <= a[1] < board_size and 0 <= b[0] < board_size and 0 <= b[1] < board_size
                    and board[a[0]][a[1]] == 'X' and board[b[0]][b[1]] == ' ' and abs(b[0]-a[0]) == 2 and abs(b[1]-a[1]) == 2
                    and (board[(a[0]+b[0])//2][(a[1]+b[1])//2] == 'o' or board[(a[0]+b[0])//2][(a[1]+b[1])//2] == 'O')):
                    return True
                else:
                    return False
            elif turn_player == 'B':
                if (0 <= a[0] < board_size and 0 <= a[1] < board_size and 0 <= b[0] < board_size and 0 <= b[1] < board_size
                    and board[a[0]][a[1]] == 'O' and board[b[0]][b[1]] == ' ' and abs(b[0]-a[0]) == 2 and abs(b[1]-a[1]) == 2
                    and (board[(a[0]+b[0])//2][(a[1]+b[1])//2] == 'x' or board[(a[0]+b[0])//2][(a[1]+b[1])//2] == 'X')):
                    return True
                else:
                    return False
        else:
            if turn_player == 'A':
                step_i = (b[0]-a[0])//abs(b[0]-a[0])
                step_j = (b[1]-a[1])//abs(b[1]-a[1])
                if (0 <= a[0] < board_size and 0 <= a[1] < board_size and 0 <= b[0] < board_size and 0 <= b[1] < board_size
                    and board[a[0]][a[1]] == 'X' and board[b[0]][b[1]] == ' ' and abs(b[0]-a[0]) == abs(b[1]-a[1])
                    and (board[b[0]-step_i][b[1]-step_j] == 'o' or board[b[0]-step_i][b[1]-step_j] == 'O')):
                    for i in range(1, abs(b[0]-2*step_i-a[0])+1):
                        if board[a[0]+i*step_i][a[1]+i*step_j] != ' ':
                            return False
                    return True
                else:
                    return False
            elif turn_player == 'B':
                step_i = (b[0]-a[0])//abs(b[0]-a[0])
                step_j = (b[1]-a[1])//abs(b[1]-a[1])
                if (0 <= a[0] < board_size and 0 <= a[1] < board_size and 0 <= b[0] < board_size and 0 <= b[1] < board_size
                    and board[a[0]][a[1]] == 'O' and board[b[0]][b[1]] == ' ' and abs(b[0]-a[0]) == abs(b[1]-a[1])
                    and (board[b[0]-step_i][b[1]-step_j] == 'x' or board[b[0]-step_i][b[1]-step_j] == 'X')):
                    for i in range(1, abs(b[0]-2*step_i-a[0])+1):
                        if board[a[0]+i*step_i][a[1]+i*step_j] != ' ':
                            return False
                    return True
                else:
                    return False
                                  
    if turn_player == 'A':
        if (0 <= a[0] < board_size and 0 <= a[1] < board_size and 0 <= b[0] < board_size and 0 <= b[1] < board_size
            and board[a[0]][a[1]] == 'x' and board[b[0]][b[1]] == ' ' and b[0]-a[0] == -2 and abs(b[1]-a[1]) == 2
            and (board[(a[0]+b[0])//2][(a[1]+b[1])//2] == 'o' or board[(a[0]+b[0])//2][(a[1]+b[1])//2] == 'O')):
            return True
        else:
            return False
    elif turn_player == 'B':     
        if (0 <= a[0] < board_size and 0 <= a[1] < board_size and 0 <= b[0] < board_size and 0 <= b[1] < board_size
            and board[a[0]][a[1]] == 'o' and board[b[0]][b[1]] == ' ' and b[0]-a[0] == 2 and abs(b[1]-a[1]) == 2
            and (board[(a[0]+b[0])//2][(a[1]+b[1])//2] == 'x' or board[(a[0]+b[0])//2][(a[1]+b[1])//2] == 'X')):
            return True
        else:
            return False
    
def is_legal_move(board, a, b, turn_player, is_now_multi_jump=False):
    if is_legal_non_jump(board, a, b, turn_player):
        return True
    elif is_legal_jump(board, a, b, turn_player, is_now_multi_jump):
        return True
    else:
        return False

def perform_move(board_param, a, b, turn_player, is_now_multi_jump=False):
    global board_size
    global previous_move
    previous_move = [a, b]
    board = deepcopy(board_param)
    if is_legal_non_jump(board, a, b, turn_player):
        board[b[0]][b[1]] = board[a[0]][a[1]]
        board[a[0]][a[1]] = ' '
    elif is_legal_jump(board, a, b, turn_player, is_now_multi_jump):
        if board[a[0]][a[1]] == 'X' or board[a[0]][a[1]] == 'O':
            board[b[0]][b[1]] = board[a[0]][a[1]]
            board[a[0]][a[1]] = ' '
            board[b[0]-((b[0]-a[0])//abs(b[0]-a[0]))][b[1]-((b[1]-a[1])//abs(b[1]-a[1]))] = ' '
        else:
            board[b[0]][b[1]] = board[a[0]][a[1]]
            board[a[0]][a[1]] = ' '
            board[(a[0]+b[0])//2][(a[1]+b[1])//2] = ' '            
    
    for i in range(board_size):
        for j in range(board_size):
            if i == 0 and board[i][j] == 'x':
                board[i][j] = 'X'
            if i == board_size-1 and board[i][j] == 'o':
                board[i][j] = 'O'
    return board

def is_unit_jump_move_exist(board, i, j, turn_player, is_now_multi_jump=False):
    global board_size
    if board[i][j] == 'X' or board[i][j] == 'O':
        if is_now_multi_jump:
            if is_legal_jump(board, [i,j], [i-2,j-2], turn_player, is_now_multi_jump):
                return True
            elif is_legal_jump(board, [i,j], [i-2,j+2], turn_player, is_now_multi_jump):
                return True
            elif is_legal_jump(board, [i,j], [i+2,j-2], turn_player, is_now_multi_jump):
                return True
            elif is_legal_jump(board, [i,j], [i+2,j+2], turn_player, is_now_multi_jump):
                return True
            return False
        else:
            x = -2
            y = -2
            while 0 <= i+x < board_size and 0 <= j+y < board_size:
                if is_legal_jump(board, [i,j], [i+x,j+y], turn_player):
                    return True
                x-=1
                y-=1
            x = -2
            y = 2
            while 0 <= i+x < board_size and 0 <= j+y < board_size:
                if is_legal_jump(board, [i,j], [i+x,j+y], turn_player):
                    return True
                x-=1
                y+=1
            x = 2
            y = -2
            while 0 <= i+x < board_size and 0 <= j+y < board_size:
                if is_legal_jump(board, [i,j], [i+x,j+y], turn_player):
                    return True
                x+=1
                y-=1
            x = 2
            y = 2
            while 0 <= i+x < board_size and 0 <= j+y < board_size:
                if is_legal_jump(board, [i,j], [i+x,j+y], turn_player):
                    return True
                x+=1
                y+=1
            return False
    if turn_player == 'A':
        if is_legal_jump(board, [i,j], [i-2,j-2], turn_player):
            return True
        elif is_legal_jump(board, [i,j], [i-2,j+2], turn_player):
            return True
    elif turn_player == 'B':
        if is_legal_jump(board, [i,j], [i+2,j-2], turn_player):
            return True
        elif is_legal_jump(board, [i,j], [i+2,j+2], turn_player):
            return True
    return False

def is_jump_move_exist(board, turn_player):
    global board_size
    if turn_player == 'A':
        for i in range(board_size):
            for j in range(board_size):
                if board[i][j] == 'x' or board[i][j] == 'X':
                    if is_unit_jump_move_exist(board, i, j, turn_player, is_now_multi_jump=False):
                        return True
    if turn_player == 'B':
        for i in range(board_size):
            for j in range(board_size):
                if board[i][j] == 'o' or board[i][j] == 'O':
                    if is_unit_jump_move_exist(board, i, j, turn_player, is_now_multi_jump=False):
                        return True
    return False
                        
def is_game_end(board):
    flatten = [item for sublist in board for item in sublist]
    if flatten.count('x') + flatten.count('X') == 0 or flatten.count('o') + flatten.count('O') == 0:
        return True
    else:
        return False

def the_winner(board):
    if not is_game_end(board):
        return
    elif flatten.count('o') == 0 and flatten.count('O') == 0 and flatten.count('x')+flatten.count('X') > 0:
        return 'A'
    elif flatten.count('x') == 0 and flatten.count('X') == 0 and flatten.count('o')+flatten.count('O') > 0:
        return 'B'

In [4]:
# continue X and O here next
def coordinate_from_button(text):
    i, j = text.split(' ')
    return int(i), int(j)

def update():
    global board
    global board_size
    global buttons
    for i in range(board_size):
        for j in range(board_size):
            if (i+j)%2 == 1:
                if board[i][j] == 'x':
                    buttons[i][j].config(image = red_piece)
                elif board[i][j] == 'o':
                    buttons[i][j].config(image = black_piece)
                elif board[i][j] == 'X':
                    buttons[i][j].config(image = king_red_piece)
                elif board[i][j] == 'O':
                    buttons[i][j].config(image = king_black_piece)
                else:
                    buttons[i][j].config(image = '')

def show_unit_color_marking(board, i, j, is_now_multi_jump=False):
    global board_size
    global buttons
    global turn_player
    buttons[i][j].config(bg='red')
    if board[i][j] == 'X' or board[i][j] == 'O':
        if is_now_multi_jump:
            if is_legal_jump(board, [i,j],[i-2,j-2],turn_player, is_now_multi_jump):
                buttons[i-2][j-2].config(bg='red')
            if is_legal_jump(board, [i,j],[i-2,j+2],turn_player, is_now_multi_jump):
                buttons[i-2][j+2].config(bg='red')
            if is_legal_jump(board, [i,j],[i+2,j-2],turn_player, is_now_multi_jump):
                buttons[i+2][j-2].config(bg='red')
            if is_legal_jump(board, [i,j],[i+2,j+2],turn_player, is_now_multi_jump):
                buttons[i+2][j+2].config(bg='red')
        elif is_unit_jump_move_exist(board, i, j, turn_player):
            x = -2
            y = -2
            while 0 <= i+x < board_size and 0 <= j+y < board_size:
                if is_legal_jump(board, [i,j], [i+x,j+y], turn_player):
                    buttons[i+x][j+y].config(bg='red')
                x-=1
                y-=1
            x = -2
            y = 2
            while 0 <= i+x < board_size and 0 <= j+y < board_size:
                if is_legal_jump(board, [i,j], [i+x,j+y], turn_player):
                    buttons[i+x][j+y].config(bg='red')
                x-=1
                y+=1
            x = 2
            y = -2
            while 0 <= i+x < board_size and 0 <= j+y < board_size:
                if is_legal_jump(board, [i,j], [i+x,j+y], turn_player):
                    buttons[i+x][j+y].config(bg='red')
                x+=1
                y-=1
            x = 2
            y = 2
            while 0 < i+x < board_size and 0 <= j+y < board_size:
                if is_legal_jump(board, [i,j], [i+x,j+y], turn_player):
                    buttons[i+x][j+y].config(bg='red')
                x+=1
                y+=1
        else:
            x = -1
            y = -1
            while 0 <= i+x < board_size and 0 <= j+y < board_size:
                if is_legal_non_jump(board, [i,j], [i+x,j+y], turn_player):
                    buttons[i+x][j+y].config(bg='red')
                x-=1
                y-=1
            x = -1
            y = 1
            while 0 <= i+x < board_size and 0 <= j+y < board_size:
                if is_legal_non_jump(board, [i,j], [i+x,j+y], turn_player):
                    buttons[i+x][j+y].config(bg='red')
                x-=1
                y+=1
            x = 1
            y = -1
            while 0 <= i+x < board_size and 0 <= j+y < board_size:
                if is_legal_non_jump(board, [i,j], [i+x,j+y], turn_player):
                    buttons[i+x][j+y].config(bg='red')
                x+=1
                y-=1
            x = 1
            y = 1
            while 0 <= i+x < board_size and 0 <= j+y < board_size:
                if is_legal_non_jump(board, [i,j], [i+x,j+y], turn_player):
                    buttons[i+x][j+y].config(bg='red')
                x+=1
                y+=1
        return
    if is_now_multi_jump or is_unit_jump_move_exist(board, i, j, turn_player):
        if turn_player == 'A':
            if is_legal_jump(board,[i,j],[i-2,j-2], turn_player):
                buttons[i-2][j-2].config(bg='red')
            if is_legal_jump(board,[i,j],[i-2,j+2], turn_player):
                buttons[i-2][j+2].config(bg='red')
        elif turn_player == 'B':
            if is_legal_jump(board,[i,j],[i+2,j-2], turn_player):
                buttons[i+2][j-2].config(bg='red')
            if is_legal_jump(board,[i,j],[i+2,j+2], turn_player):
                buttons[i+2][j+2].config(bg='red')
    else:
        if turn_player == 'A':
            if is_legal_move(board,[i,j],[i-1,j-1],turn_player):
                buttons[i-1][j-1].config(bg='red')
            if is_legal_move(board,[i,j],[i-1,j+1],turn_player):
                buttons[i-1][j+1].config(bg='red')
        elif turn_player == 'B':
            if is_legal_move(board,[i,j],[i+1,j-1],turn_player):
                buttons[i+1][j-1].config(bg='red')
            if is_legal_move(board,[i,j],[i+1,j+1],turn_player):
                buttons[i+1][j+1].config(bg='red')      
                
def show_forced_color_marking():                
    # highlight force jump units at the beginning of the turn
    global board
    global board_size
    global buttons
    global turn_player
    if is_jump_move_exist(board, turn_player):
        if turn_player == 'A':
            for i in range(board_size):
                for j in range(board_size):
                    if (board[i][j] == 'x' or board[i][j] == 'X') and is_unit_jump_move_exist(board, i, j, turn_player):
                        buttons[i][j].config(bg='pink')
        elif turn_player == 'B':
            for i in range(board_size):
                for j in range(board_size):
                    if (board[i][j] == 'o' or board[i][j] == 'O') and is_unit_jump_move_exist(board, i, j, turn_player):
                        buttons[i][j].config(bg='pink')
                        
def show_previous_move():
    global previous_move
    global buttons
    if previous_move is not None:
        buttons[previous_move[0][0]][previous_move[0][1]].config(bg='#add8e6')
        buttons[previous_move[1][0]][previous_move[1][1]].config(bg='#add8e6')
                        
def clear_color_marking():
    global board_size
    global buttons
    for i in range(board_size):
        for j in range(board_size):
            if (i+j)%2 == 1:
                buttons[i][j].config(bg='gray')

def change_turn():
    global board
    global turn_player
    if is_game_end(board):
        turn_player = 'END'
        return
    if turn_player == 'A':
        turn_player = 'B'
        board = computer_turn()
        turn_player = 'A'
        update()
    elif turn_player == 'B':
        turn_player = 'A'
    show_forced_color_marking()
    show_previous_move()

In [5]:
def recursive_jump(state_param, i, j, turn_player, possible_board_states, is_now_multi_jump):
    this_state = deepcopy(state_param)
    if not is_unit_jump_move_exist(this_state, i, j, turn_player, is_now_multi_jump):
        possible_board_states.append(this_state)
    else:
        if this_state[i][j] == 'X' or this_state[i][j] == 'O':
            if is_now_multi_jump:
                if is_legal_jump(this_state, [i,j], [i-2,j-2], turn_player, is_now_multi_jump):
                    state = perform_move(this_state, [i,j], [i-2,j-2], turn_player, is_now_multi_jump)
                    recursive_jump(state, i-2, j-2, turn_player, possible_board_states, is_now_multi_jump=True)
                if is_legal_jump(this_state, [i,j], [i-2,j+2], turn_player, is_now_multi_jump):
                    state = perform_move(this_state, [i,j], [i-2,j+2], turn_player, is_now_multi_jump)
                    recursive_jump(state, i-2, j+2, turn_player, possible_board_states, is_now_multi_jump=True)
                if is_legal_jump(this_state, [i,j], [i+2,j-2], turn_player, is_now_multi_jump):
                    state = perform_move(this_state, [i,j], [i+2,j-2], turn_player, is_now_multi_jump)
                    recursive_jump(state, i+2, j-2, turn_player, possible_board_states, is_now_multi_jump=True)
                if is_legal_jump(this_state, [i,j], [i+2,j+2], turn_player, is_now_multi_jump):
                    state = perform_move(this_state, [i,j], [i+2,j+2], turn_player, is_now_multi_jump)
                    recursive_jump(state, i+2, j+2, turn_player, possible_board_states, is_now_multi_jump=True)
            else:
                x = -2
                y = -2
                while 0 <= i+x < board_size and 0 <= j+y < board_size:
                    if is_legal_jump(this_state, [i,j], [i+x,j+y], turn_player):
                        state = perform_move(this_state, [i,j], [i+x,j+y], turn_player)
                        recursive_jump(state, i+x, j+y, turn_player, possible_board_states, is_now_multi_jump=True)
                    x-=1
                    y-=1
                x = -2
                y = 2
                while 0 <= i+x < board_size and 0 <= j+y < board_size:
                    if is_legal_jump(this_state, [i,j], [i+x,j+y], turn_player):
                        state = perform_move(this_state, [i,j], [i+x,j+y], turn_player)
                        recursive_jump(state, i+x, j+y, turn_player, possible_board_states, is_now_multi_jump=True)
                    x-=1
                    y+=1
                x = 2
                y = -2
                while 0 <= i+x < board_size and 0 <= j+y < board_size:
                    if is_legal_jump(this_state, [i,j], [i+x,j+y], turn_player):
                        state = perform_move(this_state, [i,j], [i+x,j+y], turn_player)
                        recursive_jump(state, i+x, j+y, turn_player, possible_board_states, is_now_multi_jump=True)
                    x+=1
                    y-=1
                x = 2
                y = 2
                while 0 <= i+x < board_size and 0 <= j+y < board_size:
                    if is_legal_jump(this_state, [i,j], [i+x,j+y], turn_player):
                        state = perform_move(this_state, [i,j], [i+x,j+y], turn_player)
                        recursive_jump(state, i+x, j+y, turn_player, possible_board_states, is_now_multi_jump=True)
                    x+=1
                    y+=1             
        elif this_state[i][j] == 'x':
            if is_legal_jump(this_state, [i,j], [i-2,j-2], turn_player, is_now_multi_jump):
                state = perform_move(this_state, [i,j], [i-2,j-2], turn_player, is_now_multi_jump)
                recursive_jump(state, i-2, j-2, turn_player, possible_board_states, is_now_multi_jump=True)
            if is_legal_jump(this_state, [i,j], [i-2,j+2], turn_player, is_now_multi_jump):
                state = perform_move(this_state, [i,j], [i-2,j+2], turn_player, is_now_multi_jump)
                recursive_jump(state, i-2, j+2, turn_player, possible_board_states, is_now_multi_jump=True)
        elif this_state[i][j] == 'o':
            if is_legal_jump(this_state, [i,j], [i+2,j-2], turn_player, is_now_multi_jump):
                state = perform_move(this_state, [i,j], [i+2,j-2], turn_player, is_now_multi_jump)
                recursive_jump(state, i+2, j-2, turn_player, possible_board_states, is_now_multi_jump=True)
            if is_legal_jump(this_state, [i,j], [i+2,j+2], turn_player, is_now_multi_jump):
                state = perform_move(this_state, [i,j], [i+2,j+2], turn_player, is_now_multi_jump)
                recursive_jump(state, i+2, j+2, turn_player, possible_board_states, is_now_multi_jump=True)
    
def all_possible_moves(board, turn_player):
    global board_size
    possible_board_states = []
    if is_jump_move_exist(board, turn_player):
        if turn_player == 'A':
            for i in range(board_size):
                for j in range(board_size):
                    if board[i][j] == 'X' or board[i][j] == 'x':
                        if is_unit_jump_move_exist(board, i, j, turn_player):
                            recursive_jump(board, i, j, turn_player, possible_board_states, is_now_multi_jump=False)
        elif turn_player == 'B':
            for i in range(board_size):
                for j in range(board_size):
                    if board[i][j] == 'O' or board[i][j] == 'o':
                        if is_unit_jump_move_exist(board, i, j, turn_player):
                            recursive_jump(board, i, j, turn_player, possible_board_states, is_now_multi_jump=False)
    else:
        if turn_player == 'A':
            for i in range(board_size):
                for j in range(board_size):
                    if board[i][j] == 'X':
                        x = -1
                        y = -1
                        while 0 <= i+x < board_size and 0 <= j+y < board_size:
                            if is_legal_non_jump(board, [i,j], [i+x,j+y], turn_player):
                                state = perform_move(board, [i,j], [i+x,j+y], turn_player)
                                possible_board_states.append(state)
                            x-=1
                            y-=1
                        x = -1
                        y = 1
                        while 0 <= i+x < board_size and 0 <= j+y < board_size:
                            if is_legal_non_jump(board, [i,j], [i+x,j+y], turn_player):
                                state = perform_move(board, [i,j], [i+x,j+y], turn_player)
                                possible_board_states.append(state)
                            x-=1
                            y+=1
                        x = 1
                        y = -1
                        while 0 <= i+x < board_size and 0 <= j+y < board_size:
                            if is_legal_non_jump(board, [i,j], [i+x,j+y], turn_player):
                                state = perform_move(board, [i,j], [i+x,j+y], turn_player)
                                possible_board_states.append(state)
                            x+=1
                            y-=1
                        x = 1
                        y = 1
                        while 0 <= i+x < board_size and 0 <= j+y < board_size:
                            if is_legal_non_jump(board, [i,j], [i+x,j+y], turn_player):
                                state = perform_move(board, [i,j], [i+x,j+y], turn_player)
                                possible_board_states.append(state)
                            x+=1
                            y+=1
                    elif board[i][j] == 'x':
                        if is_legal_non_jump(board, [i,j], [i-1,j-1], turn_player):
                            state = perform_move(board, [i,j], [i-1,j-1], turn_player)
                            possible_board_states.append(state)
                        if is_legal_non_jump(board, [i,j], [i-1,j+1], turn_player):
                            state = perform_move(board, [i,j], [i-1,j+1], turn_player)
                            possible_board_states.append(state)
        elif turn_player == 'B':
            for i in range(board_size):
                for j in range(board_size):
                    if board[i][j] == 'O':
                        x = -1
                        y = -1
                        while 0 <= i+x < board_size and 0 <= j+y < board_size:
                            if is_legal_non_jump(board, [i,j], [i+x,j+y], turn_player):
                                state = perform_move(board, [i,j], [i+x,j+y], turn_player)
                                possible_board_states.append(state)
                            x-=1
                            y-=1
                        x = -1
                        y = 1
                        while 0 <= i+x < board_size and 0 <= j+y < board_size:
                            if is_legal_non_jump(board, [i,j], [i+x,j+y], turn_player):
                                state = perform_move(board, [i,j], [i+x,j+y], turn_player)
                                possible_board_states.append(state)
                            x-=1
                            y+=1
                        x = 1
                        y = -1
                        while 0 <= i+x < board_size and 0 <= j+y < board_size:
                            if is_legal_non_jump(board, [i,j], [i+x,j+y], turn_player):
                                state = perform_move(board, [i,j], [i+x,j+y], turn_player)
                                possible_board_states.append(state)
                            x+=1
                            y-=1
                        x = 1
                        y = 1
                        while 0 <= i+x < board_size and 0 <= j+y < board_size:
                            if is_legal_non_jump(board, [i,j], [i+x,j+y], turn_player):
                                state = perform_move(board, [i,j], [i+x,j+y], turn_player)
                                possible_board_states.append(state)
                            x+=1
                            y+=1
                    elif board[i][j] == 'o':
                        if is_legal_non_jump(board, [i,j], [i+1,j-1], turn_player):
                            state = perform_move(board, [i,j], [i+1,j-1], turn_player)
                            possible_board_states.append(state)
                        if is_legal_non_jump(board, [i,j], [i+1,j+1], turn_player):
                            state = perform_move(board, [i,j], [i+1,j+1], turn_player)
                            possible_board_states.append(state)
    return possible_board_states

In [6]:
def on_click(self):
    global board
    global board_size
    global is_selected_active
    global current_selection
    global turn_player
    global is_now_multi_jump
    global multi_jump_coor
    global buttons
    
    i, j = coordinate_from_button(self['text'])
    
    # continue multiple jump (forced) case
    if turn_player == 'END':
        return
    
    if is_now_multi_jump:
        if is_legal_jump(board, multi_jump_coor, [i,j], turn_player, is_now_multi_jump):
            board = perform_move(board, multi_jump_coor, [i,j], turn_player, is_now_multi_jump)
            update()
            multi_jump_coor = [i, j]
            clear_color_marking()
            if is_unit_jump_move_exist(board, i, j, turn_player, is_now_multi_jump):
                show_unit_color_marking(board, i, j, is_now_multi_jump)
            else:
                is_now_multi_jump = False
                multi_jump_coor = None
                change_turn()
            return
        else:
            return
    
    # force jump case 
    if is_jump_move_exist(board, turn_player):
        if not is_selected_active and not is_unit_jump_move_exist(board, i, j, turn_player):
            return
        elif not is_selected_active:
            is_selected_active = True
            current_selection = [i, j]
            show_unit_color_marking(board, i, j)
            return
        elif is_selected_active:
            if [i, j] == current_selection:
                is_selected_active = False
                current_selection = None
                clear_color_marking()
                show_forced_color_marking()
                return
            elif [i, j] != current_selection:
                if is_legal_jump(board, current_selection, [i,j], turn_player):
                    board = perform_move(board, current_selection, [i,j], turn_player)
                    update()
                    clear_color_marking()
                    is_selected_active = False
                    current_selection = None
                    if is_unit_jump_move_exist(board, i, j, turn_player, is_now_multi_jump):
                        is_now_multi_jump = True
                        multi_jump_coor = [i, j]
                        show_unit_color_marking(board, i, j)
                    else:
                        change_turn()
                return
 
    # non-jump case
    if not is_selected_active:
        if turn_player == 'A' and not (board[i][j] == 'x' or board[i][j] == 'X'):
            return
        elif turn_player == 'B' and not (board[i][j] == 'o' or board[i][j] == 'O'):
            return
    if not is_selected_active:
        is_selected_active = True
        current_selection = [i, j]
        show_unit_color_marking(board, i, j)
        return
    elif is_selected_active:
        if [i, j] == current_selection:
            is_selected_active = False
            current_selection = None
            clear_color_marking()
            return
        elif [i, j] != current_selection:
            if is_legal_non_jump(board, current_selection, [i,j], turn_player):
                board = perform_move(board, current_selection, [i,j], turn_player)
                update()
                clear_color_marking()
                is_selected_active = False
                current_selection = None
                change_turn()
                return
            else:
                return

In [10]:
def heuristic_value(board):
    global board_size
    flatten = [item for sublist in board for item in sublist]
    value = flatten.count('o') + 3*flatten.count('O') - flatten.count('x') - 3*flatten.count('X')
    return value

def minimax(board, depth, maximizing_player):
    if depth == 0 or is_game_end(board):
        return heuristic_value(board)
    if maximizing_player == 'B':
        max_val = -math.inf
        for state in all_possible_moves(board, maximizing_player):
            value = minimax(state, depth-1, maximizing_player='A')
            max_val = max(max_val, value)
        return max_val
    elif maximizing_player == 'A':
        min_val = math.inf
        for state in all_possible_moves(board, maximizing_player):
            value = minimax(state, depth-1, maximizing_player='B')
            min_val = min(min_val, value)
        return min_val

def alpha_beta(board_param, depth, alpha, beta, maximizing_player):
    if depth == 0 or is_game_end(board_param):
        return heuristic_value(board_param)
    if maximizing_player == 'B':
        max_val = -math.inf
        for state in all_possible_moves(board_param, maximizing_player):
            value = alpha_beta(state, depth-1, alpha, beta, maximizing_player='A')
            max_val = max(max_val, value)
            alpha = max(alpha, max_val)
            if alpha >= beta:
                break
        return max_val
    elif maximizing_player == 'A':
        min_val = math.inf
        for state in all_possible_moves(board_param, maximizing_player):
            value = alpha_beta(state, depth-1, alpha, beta, maximizing_player='B')
            min_val = min(min_val, value)
            beta = min(beta, min_val)
            if alpha >= beta:
                break
        return min_val

def computer_turn():
    global board
    global turn_player
    all_moves_scores = []
    for state in all_possible_moves(board, turn_player):
        #all_moves_scores.append([minimax(state, depth=3, maximizing_player='A'), state])
        all_moves_scores.append([alpha_beta(state, depth=3, alpha=-math.inf, beta=math.inf, maximizing_player='A'), state])
    max_val = -math.inf
    for x in all_moves_scores:
        max_val = max(max_val, x[0])
    best_states = [x for x in all_moves_scores if x[0] == max_val]
    pick_state = random.choice(best_states)
    #for idx, state in all_moves_scores:
    #    print('value =', idx)
    #    print_board(state)
    return pick_state[1]

In [11]:
# global variables
board_size = 8
board = None
unit_rows = 2

turn_player = 'A'
is_selected_active = False
current_selection = None
is_game_end1 = False
is_now_multi_jump = False
multi_jump_coor = None
previous_move = None
possible_board_states = []

board = init_board()
print_board(board)

  0 1 2 3 4 5 6 7
0 - o - o - o - o
1 o - o - o - o -
2 -   -   -   -  
3   -   -   -   -
4 -   -   -   -  
5   -   -   -   -
6 - x - x - x - x
7 x - x - x - x -


In [ ]:
buttons = [[None]*board_size for _ in range(board_size)]

root = tk.Tk()

red_piece = ImageTk.PhotoImage(Image.open('red_piece.png').resize((100, 100)))
black_piece = ImageTk.PhotoImage(Image.open('black_piece.png').resize((100, 100)))
king_red_piece = ImageTk.PhotoImage(Image.open('king_red_piece.png').resize((100, 100)))
king_black_piece = ImageTk.PhotoImage(Image.open('king_black_piece.png').resize((100, 100)))

for i in range(board_size):
    for j in range(board_size):
        if (i+j)%2 == 1:
            buttons[i][j] = tk.Button(root, text=str(i)+' '+str(j), height=5, width=10, state='normal', bg='gray', fg='gray')
        else:
            buttons[i][j] = tk.Button(root, text=str(i)+' '+str(j), height=5, width=10, state='disabled', bg='white', fg='white')

for i in range(board_size):
    for j in range(board_size):
        if (i+j)%2 == 1:
            if board[i][j] == 'x':
                buttons[i][j].config(image = red_piece, command=lambda arg=buttons[i][j]:on_click(arg))
            elif board[i][j] == 'o':
                buttons[i][j].config(image = black_piece, command=lambda arg=buttons[i][j]:on_click(arg))
            elif board[i][j] == 'X':
                buttons[i][j].config(image = king_red_piece, command=lambda arg=buttons[i][j]:on_click(arg))
            elif board[i][j] == 'O':
                buttons[i][j].config(image = king_black_piece, command=lambda arg=buttons[i][j]:on_click(arg))
            else:
                buttons[i][j].config(command=lambda arg=buttons[i][j]:on_click(arg))

for i, row in enumerate(buttons):
    for j, element in enumerate(row):
        buttons[i][j].grid(row=i, column=j, sticky='news')

root.mainloop()